In [10]:
import requests
import sys
import pandas as pd
from bs4 import BeautifulSoup
import os
import csv

def met_data(month, year):
    
    file_html = open('Data/Html_Data/{}/{}.html'.format(year,month), 'rb')
    plain_text = file_html.read()

    tempD = []
    finalD = []

    soup = BeautifulSoup(plain_text, "lxml")
    for table in soup.findAll('table', {'class': 'medias mensuales numspan'}):
        for tbody in table:
            for tr in tbody:
                a = tr.get_text()
                tempD.append(a)

    rows = len(tempD) / 15

    for times in range(round(rows)):
        newtempD = []
        for i in range(15):
            newtempD.append(tempD[0])
            tempD.pop(0)
        finalD.append(newtempD)

    length = len(finalD)

    finalD.pop(length - 1)
    finalD.pop(0)


    for a in range(len(finalD)):
        finalD[a].pop(14)
        finalD[a].pop(13)
        finalD[a].pop(12)
        finalD[a].pop(11)
        finalD[a].pop(10)
        finalD[a].pop(6)
        finalD[a].pop(4)
        finalD[a].pop(0)
        

    
    return finalD

def data_combine(year, cs):
    for a in pd.read_csv('Data/Real-Data/real_' + str(year) + '.csv', chunksize=cs):
        df = pd.DataFrame(data=a)
        mylist = df.values.tolist()
    return mylist


if __name__ == "__main__":
    if not os.path.exists("Data/Real-Data"):
        os.makedirs("Data/Real-Data")
    for year in range(2015, 2020):
        final_data = []
        with open('Data/Real-Data/real_' + str(year) + '.csv', 'w') as csvfile:
            wr = csv.writer(csvfile, dialect='excel')
            wr.writerow(
                ['T', 'TM', 'Tm', 'H', 'VV', 'V', 'VM', 'PM 2.5'])
        for month in range(1, 13):
            temp = met_data(month, year)
            final_data = final_data + temp
            
        x = open('{}.csv'.format(year))
        pm = []
        for i in x:
            pm.append(i.strip())
        print('extracting year : ', year)

        if len(pm) == 364:
            pm.insert(364, '-')

        for i in range(len(final_data)-1):
            # final[i].insert(0, i + 1)
            final_data[i].insert(7, pm[i])
            #print(final_data)

        with open('Data/Real-Data/real_' + str(year) + '.csv', 'a') as csvfile:
            wr = csv.writer(csvfile, dialect='excel')
            for row in final_data:
                flag = 0
                for elem in row:
                    if elem == "" or elem == "-":
                        flag = 1
                if flag != 1:
                    wr.writerow(row)
                    
    data_2015 = data_combine(2015, 600)
    data_2016 = data_combine(2016, 600)
    data_2017 = data_combine(2017, 600)
    data_2018 = data_combine(2018, 600)
    data_2019 = data_combine(2019, 600)
     
    total = data_2015 + data_2016 + data_2017 + data_2018 + data_2019
    
    with open('Data/Real-Data/Real_Combine.csv', 'w') as csvfile:
        wr = csv.writer(csvfile, dialect='excel')
        wr.writerow(
            ['T', 'TM', 'Tm', 'H', 'VV', 'V', 'VM', 'PM 2.5'])
        wr.writerows(total)
        print()
        print('Done with Dataset.... Let us move to the machine learning part! ')
        
        
df = pd.read_csv('Data/Real-Data/Real_Combine.csv')

extracting year :  2015
extracting year :  2016
extracting year :  2017
extracting year :  2018
extracting year :  2019

Done with Dataset.... Let us move to the machine learning part! 


In [4]:
print(sum(df.isnull().any()))

1


In [2]:
df.tail()

,T,TM,Tm,H,VV,V,VM,PM 2.5
921,12.2,16.4,8.9,93.0,0.6,8.0,13.0,249.87
922,8.2,15.0,5.4,97.0,0.6,11.3,14.8,166.71
923,8.0,11.6,5.6,90.0,0.8,7.8,13.0,196.97
924,8.6,12.0,4.2,84.0,0.8,4.6,11.1,240.07
925,9.0,14.0,5.0,80.0,0.6,4.1,13.0,NaN
